<p style="text-align: center; font-size: 300%"> Computational Finance </p>
<img src="img/ABSlogo.svg" alt="LOGO" style="display:block; margin-left: auto; margin-right: auto; width: 50%;">

# Advanced Monte Carlo Methods
## Variance Reduction Techniques
* In standard Monte Carlo, the length of the confidence interval for $\theta\equiv\mathbb{E}[X]$ is proportional to $\hat{\sigma}/\sqrt{n}$, where $\sigma$ is the standard deviation of $X$.
* Thus to increase the accuracy by a factor 10 (i.e., gain 1 digit), we need 100 times as many samples.
* Variance reduction techniques aim to improve the accuracy of the estimate, without increasing $n$.
* We will consider two such techniques: *antithetic sampling*, and *control variates*.
* Another powerful technique is *importance sampling*, but this is beyond the scope of this course.

## Antithetic Sampling
* The crude MC estimate for $\theta\equiv \mathbb{E} [X]$, based on $n$ independent draws $X_i$, is
$$
\hat{\theta}=\frac{1}{n}\sum_{i=1}^n X_i,
$$
* Now suppose that we can somehow sample $n$ pairs $(X_i, \tilde X_i)$, such that
  * both $X_i$ and $\tilde X_i$ are drawn from the distribution of $X$;
  * the *pairs* $(X_i, \tilde X_i)$ are independent across $i$;
  * for each $i$, $X_i$ and $\tilde X_i$ are (negatively) correlated.
* The antithetic variable estimator is then
$$
\hat{\theta}_{AV}=\frac{1}{2}\left(\frac{1}{n}\sum_{i=1}^n X_i+\frac{1}{n}\sum_{i=1}^n \tilde X_i\right).
$$

* Rewriting the estimator as
$$
\hat{\theta}_{AV}=\frac{1}{n}\sum_{i=1}^n\left(\frac{X_i+\tilde X_i}{2}\right),
$$
we see that it is unbiased, and that $\hat{\theta}_{AV}$ is the mean of $n$ independent observations, $(X_i+\tilde X_i)/2$, so that the LLN and CLT continue to apply.
* Hence, by the CLT,
$$
\sqrt{n}(\hat{\theta}_{AV}-\theta)\stackrel d \rightarrow N(0,\sigma^2_{AV}),
$$
where
\begin{align*}
\sigma^2_{AV}&=\mathrm{Var}\left[\frac{X_i+\tilde X_i}{2}\right]=\frac{1}{4}\left(\mathrm{Var}[X_i]+\mathrm{Var}[\tilde X_i] +2\mathrm{Cov}[X_i,\tilde X_i]\right)\\
%&=\frac{1}{2}\left[\mathrm{Var}[X_i] +\mathrm{Cov}[X_i,\tilde X_i]\right)
&=\frac{\sigma^2}{2}\big(1+\rho(X_i,\tilde X_i)\big).
\end{align*}

* Comparing the variance of $\hat{\theta}_{AV}$, $\frac{1}{n}\sigma^2_{AV}$, with that of the crude estimator based on $2n$ observations $\left(\frac{1}{2n}\sigma^2\right)$, we see that efficiency is gained whenever the correlation $\rho(X_i,\tilde X_i)$ is negative.
* So how do we draw correlated random numbers $X_i$ and $\tilde X_i$?
* Our simulations are typically based on an array of standard normal random numbers $\mathbf{z}_i$, i.e., $X_i=f(\mathbf{z}_i)$. Setting $\tilde X_i=f(-\mathbf{z}_i)$ will often do the trick. 
* Note for standard Brownian motion, this corresponds to flipping the path about the abscissa.

In [1]:
import numpy as np
from scipy.stats import norm

In [2]:
def bmsim_vec(T, N, X0=0, mu=0, sigma=1, numsim=1, av=False):
    """Simulate `numsim` Brownian motion paths. If av=True, then 2*`numsim` paths are returned,
    where paths numsim:2numsim+1 are antithetic paths.
    """
    deltaT = float(T)/N
    tvec = np.linspace(0, T, N+1)
    z = np.random.randn(numsim, N+1)
    if av:
        z=np.concatenate((z, -z))
    dX = mu*deltaT + sigma*np.sqrt(deltaT)*z
    dX[:, 0] = 0.
    X = np.cumsum(dX, axis=1)
    X += X0    
    return tvec, X

In [3]:
def asianmc_vec(S0, K, T, r, sigma, delta, N, numsim=1000, av=False):
    """
    Monte Carlo price of an arithmetic average Asian call.
    """
    X0 = np.log(S0)
    nu = r-delta-.5*sigma**2    
    _, X = bmsim_vec(T, N, X0, nu, sigma, numsim, av)
    S = np.exp(X)
    payoffs = np.maximum(S[:, 1:].mean(axis=1)-K, 0.)
    g = np.exp(-r*T)*payoffs
    if av:
        g=.5*(g[:numsim]+g[numsim:])    
    C = g.mean();s = g.std()
    zq = norm.ppf(0.975)
    Cl = C - zq/np.sqrt(numsim)*s
    Cu = C + zq/np.sqrt(numsim)*s
    return C, Cl, Cu

In [4]:
S0=11; K=10; T=3/12.; r=0.02; sigma=.3; delta=0.01; N=10; numsim=10**4
np.random.seed(0)
C, Cl, Cu=asianmc_vec(S0, K, T, r, sigma, delta, N, numsim, False)
C, Cu-Cl

(1.0927262054551385, 0.03592162508407748)

In [5]:
np.random.seed(0)
C, Cl, Cu=asianmc_vec(S0, K, T, r, sigma, delta, N, numsim/2, True)
C, Cu-Cl

(1.0824190804865295, 0.012604081385357624)